In [11]:
import os
import random
from tqdm import tqdm
import numpy as np
from PIL import Image
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split, SubsetRandomSampler
from sklearn.metrics import accuracy_score, pairwise_distances
from sklearn.cluster import KMeans

torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

## Data Loader

In [12]:
def load_and_preprocess_image(image_path, size=(64, 64)):
    """
    Lädt ein Bild von einem gegebenen Pfad, ändert dessen Größe und konvertiert es in Graustufen.
    """
    image = Image.open(image_path)
    image = image.resize(size).convert('L')
    image_array = np.array(image) / 255.0
    
    return image_array

def load_and_preprocess_images(directory, csv_path, size=(64, 64)):
    """
    Lädt und verarbeitet alle Bilder in einem Verzeichnis basierend auf einer CSV-Datei, die Labels enthält.
    """
    # CSV-Datei laden, die Labels und Bildpfade enthält
    labels_df = pd.read_csv(csv_path)
    
    images = []
    labels = []
    
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        
        if os.path.isfile(file_path) and filename.lower().endswith(('.png', '.jpg', '.jpeg')):

            image_array = load_and_preprocess_image(file_path, size)
            images.append(image_array)
            # Das Label aus der CSV-Datei extrahieren
            label = labels_df[labels_df['image'] == filename]['labels'].values[0]
            labels.append(label)
    
    return np.array(images), np.array(labels)


directory = 'dataset/cat_dog'
csv_path = 'dataset/cat_dog.csv'

images, labels = load_and_preprocess_images(directory, csv_path)

print(f"Anzahl geladener Bilder: {len(images)}")
print(f"Form eines Bildes: {images[0].shape}")
print(f"Erste 5 Labels: {labels[:5]}")

Anzahl geladener Bilder: 25000
Form eines Bildes: (64, 64)
Erste 5 Labels: [1 0 1 0 0]


## Model definition (simple cnn)

In [13]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)  # 16 Filter, Kernel-Größe 3
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1) # 32 Filter, Kernel-Größe 3
        self.fc1 = nn.Linear(32 * 16 * 16, 128)      # Vollverbundene Schicht
        self.fc2 = nn.Linear(128, 2)                 # Ausgangsschicht
        self.pool = nn.MaxPool2d(2, 2)               # Pooling-Schicht
        self.relu = nn.ReLU()                        # ReLU Aktivierungsfunktion

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 32 * 16 * 16)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

print(model)

SimpleCNN(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=8192, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu): ReLU()
)


## Aufteilung der Daten und Umwandlung in Tensoren

In [14]:
tensor_images = torch.tensor(images).float()
tensor_labels = torch.tensor(labels).long()

total_count = len(tensor_labels)

num_labeled = int(0.25 * total_count)
num_test = int(0.1 * num_labeled)

# Indizes mischen und aufteilen
indices = torch.randperm(total_count)
labeled_indices = indices[:num_labeled]
test_indices = labeled_indices[:num_test]  # Testindizes von den gelabelten Indizes nehmen
train_indices = labeled_indices[num_test:]  # Restliche gelabelte Daten für Training
unlabeled_indices = indices[num_labeled:]

# Erstellen von Datenladern
test_dataset = TensorDataset(tensor_images[test_indices], tensor_labels[test_indices])
train_dataset = TensorDataset(tensor_images[train_indices], tensor_labels[train_indices])
# unlabeled_dataset = TensorDataset(tensor_images[unlabeled_indices], torch.zeros(len(unlabeled_indices))) # Keine Labels für ungelabelte Daten -> für manuelles Labeln
unlabeled_dataset = TensorDataset(tensor_images[unlabeled_indices], tensor_labels[unlabeled_indices]) # Für simulation des human-in-the-loop

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)
unlabeled_loader = DataLoader(unlabeled_dataset, batch_size=64, shuffle=True)

print(f"Anzahl Trainingsdaten: {len(train_loader.dataset)}")
print(f"Anzahl Testdaten: {len(test_loader.dataset)}")
print(f"Anzahl 'ungelabelter' Daten: {len(unlabeled_loader.dataset)}")

Anzahl Trainingsdaten: 5625
Anzahl Testdaten: 625
Anzahl 'ungelabelter' Daten: 18750


## Initiales Training als Vergleichsbasis

In [15]:
def train(model, data_loader, criterion, optimizer, epochs):
    model.train()
    for epoch in range(epochs):
        progress_bar = tqdm(data_loader, desc=f"Epoch {epoch+1}")
        for images, labels in progress_bar:
            # Format (batch_size, channels, width, height)
            images = images.unsqueeze(1)  # Dimension für den Kanal
            
            # Feed Forward
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # Backprop.
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            progress_bar.set_postfix(loss=loss.item())

train(model, train_loader, criterion, optimizer, epochs=20)
torch.save(model.state_dict(), 'init_model.pth')

Epoch 20: 100%|██████████| 88/88 [00:06<00:00, 14.54it/s, loss=0.0321] 


In [16]:
def evaluate(model, data_loader):
    model.eval()
    predictions, true_labels = [], []
    
    with torch.no_grad():
        for images, labels in data_loader:
            images = images.unsqueeze(1)  # Dimension für den Kanal hinzufügen
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            predictions.extend(predicted.numpy())
            true_labels.extend(labels.numpy())

    accuracy = accuracy_score(true_labels, predictions)
    return accuracy

test_accuracy = evaluate(model, test_loader)
print(f"Genauigkeit auf Testdaten: {test_accuracy}")

Genauigkeit auf Testdaten: 0.6928


## Uncertainty Sampling

gewählte Methoden 

1. Entropie 
2. Lowest Confidence.

In [17]:
def predict_uncertainty(model, data_loader):
    model.eval()
    uncertainties = []
    with torch.no_grad():
        for images, _ in data_loader:
            images = images.unsqueeze(1)  # Dimension für den Kanal hinzufügen
            outputs = model(images)
            probabilities = F.softmax(outputs, dim=1)
            entropy = -torch.sum(probabilities * torch.log(probabilities), dim=1)
            uncertainties.extend(entropy.tolist())
    return uncertainties

def predict_confidence(model, data_loader):
    model.eval()
    confidences = []
    with torch.no_grad():
        for images, _ in data_loader:
            images = images.unsqueeze(1)  # Dimension für den Kanal hinzufügen
            outputs = model(images)
            probabilities = F.softmax(outputs, dim=1)
            max_confidence, _ = torch.max(probabilities, dim=1)
            confidences.extend((1 - max_confidence).tolist())
    return confidences

def active_learning_cycle(unlabeled_loader_setting, train_loader, test_loader, criterion, optimizer, num_iterations=10, num_to_label=50, uncertainty_method="entropy"):

    model = SimpleCNN()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    for iteration in range(num_iterations):
        
        # Berechne Unsicherheiten
        if uncertainty_method == "entropy":
            uncertainties = predict_uncertainty(model, unlabeled_loader_setting)
        elif uncertainty_method == "confidence":
            uncertainties = predict_confidence(model, unlabeled_loader_setting)
        else:
            raise ValueError("Ungültige Uncertainty-Methode")
        
        uncertain_indices = sorted(range(len(uncertainties)), key=lambda i: uncertainties[i], reverse=True)[:num_to_label]
        
        # Trainingsdaten mit gesampelten Daten kombinieren
        new_images = torch.stack([unlabeled_loader_setting.dataset[i][0] for i in uncertain_indices])
        new_labels = torch.tensor([unlabeled_loader_setting.dataset[i][1] for i in uncertain_indices])
        combined_images = torch.cat((train_loader.dataset.tensors[0], new_images), dim=0)
        combined_labels = torch.cat((train_loader.dataset.tensors[1], new_labels), dim=0)
        combined_dataset = TensorDataset(combined_images, combined_labels)
        combined_loader = DataLoader(combined_dataset, batch_size=64, shuffle=True)
        
        # Training
        train(model, combined_loader, criterion, optimizer, epochs=2)
        
        # Eval.
        test_accuracy = evaluate(model, test_loader)
        print(f"Iteration {iteration + 1}, Genauigkeit auf Testdaten: {test_accuracy:.2f}")
    
    return model

# Uncertainty Sampling mit Entropie
model_entropy = active_learning_cycle(unlabeled_loader, train_loader, test_loader, criterion, optimizer, uncertainty_method="entropy")
torch.save(model_entropy.state_dict(), 'uncertain_entropy_model.pth')

# Uncertainty Sampling mit Konfidenzwerten
model_confidence = active_learning_cycle(unlabeled_loader, train_loader, test_loader, criterion, optimizer, uncertainty_method="confidence")
torch.save(model_confidence.state_dict(), 'uncertain_confidence_model.pth')

# Vergleich
accuracy_entropy = evaluate(model_entropy, test_loader)
accuracy_confidence = evaluate(model_confidence, test_loader)

print(f"Genauigkeit mit Entropie-Unsicherheit: {accuracy_entropy:.2f}")
print(f"Genauigkeit mit Konfidenz-Unsicherheit: {accuracy_confidence:.2f}")

Epoch 2: 100%|██████████| 89/89 [00:06<00:00, 14.74it/s, loss=0.681]


Iteration 1, Genauigkeit auf Testdaten: 0.63


Epoch 2: 100%|██████████| 89/89 [00:06<00:00, 14.23it/s, loss=0.616]


Iteration 2, Genauigkeit auf Testdaten: 0.71


Epoch 2: 100%|██████████| 89/89 [00:06<00:00, 14.05it/s, loss=0.521]


...

Iteration 8, Genauigkeit auf Testdaten: 0.71


Epoch 2: 100%|██████████| 89/89 [00:06<00:00, 13.57it/s, loss=0.0147] 


Iteration 9, Genauigkeit auf Testdaten: 0.70


Epoch 2: 100%|██████████| 89/89 [00:06<00:00, 14.05it/s, loss=0.0106] 


Iteration 10, Genauigkeit auf Testdaten: 0.71
Genauigkeit mit Entropie-Unsicherheit: 0.72
Genauigkeit mit Konfidenz-Unsicherheit: 0.71


## diversity sampling

Methoden implementiert:

1. Cluster-based Sampling

2. Farthest-first Traversal (größter Abstand zwischen Merkmalen, berechnet mithilfe von `pairwise_distances`)

In [19]:
def diversity_sampling_cluster(model, unlabelled_loader, num_to_label):
    model.eval()
    features = []
    with torch.no_grad():
        for images, _ in unlabelled_loader:
            images = images.unsqueeze(1)  # Dimension für den Kanal hinzufügen
            outputs = model(images)
            features.extend(outputs.tolist())
    
    kmeans = KMeans(n_clusters=num_to_label, random_state=0).fit(features)
    cluster_centers = kmeans.cluster_centers_
    
    diverse_indices = []
    for center in cluster_centers:
        distances = pairwise_distances([center], features)[0]
        closest_index = distances.argmin()
        diverse_indices.append(closest_index)
    
    return diverse_indices

def diversity_sampling_farthest(model, unlabelled_loader, num_to_label):
    model.eval()
    features = []
    with torch.no_grad():
        for images, _ in unlabelled_loader:
            images = images.unsqueeze(1)  # Dimension für den Kanal hinzufügen
            outputs = model(images)
            features.extend(outputs.tolist())
    
    diverse_indices = [0]  # Startpunkt f. berechnung der distanzen
    for _ in range(num_to_label - 1):
        distances = pairwise_distances([features[diverse_indices[-1]]], features)[0]
        farthest_index = distances.argmax()
        diverse_indices.append(farthest_index)
    
    return diverse_indices

def active_learning_cycle(unlabeled_loader_setting, train_loader, test_loader, criterion, optimizer, num_iterations=10, num_to_label=50, diversity_method="cluster"):

    model = SimpleCNN()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    for iteration in range(num_iterations):
        
        # Berechne Diversität
        if diversity_method == "cluster":
            diverse_indices = diversity_sampling_cluster(model, unlabeled_loader_setting, num_to_label)
        elif diversity_method == "farthest":
            diverse_indices = diversity_sampling_farthest(model, unlabeled_loader_setting, num_to_label)
        else:
            raise ValueError("Ungültige Diversity-Methode")
        
        # Trainingsdaten mit gesampelten Daten kombinieren
        new_images = torch.stack([unlabeled_loader_setting.dataset[i][0] for i in diverse_indices])
        new_labels = torch.tensor([unlabeled_loader_setting.dataset[i][1] for i in diverse_indices])
        
        combined_images = torch.cat((train_loader.dataset.tensors[0], new_images), dim=0)
        combined_labels = torch.cat((train_loader.dataset.tensors[1], new_labels), dim=0)
        combined_dataset = TensorDataset(combined_images, combined_labels)
        combined_loader = DataLoader(combined_dataset, batch_size=64, shuffle=True)
        
        # Training
        train(model, combined_loader, criterion, optimizer, epochs=2)
        
        # Eval.
        test_accuracy = evaluate(model, test_loader)
        print(f"Iteration {iteration + 1}, Genauigkeit auf Testdaten: {test_accuracy}")
    
    return model

# Diversity Sampling mit Cluster-based Sampling
model_cluster = active_learning_cycle(unlabeled_loader, train_loader, test_loader, criterion, optimizer, diversity_method="cluster")
torch.save(model_cluster.state_dict(), 'diversity_cluster_model.pth')

# Diversity Sampling mit Farthest-first Traversal
model_farthest = active_learning_cycle(unlabeled_loader, train_loader, test_loader, criterion, optimizer, diversity_method="farthest")
torch.save(model_farthest.state_dict(), 'diversity_farthest_model.pth')

# Vergleich
accuracy_cluster = evaluate(model_cluster, test_loader)
accuracy_farthest = evaluate(model_farthest, test_loader)

print(f"Genauigkeit mit Cluster-based Sampling: {accuracy_cluster}")
print(f"Genauigkeit mit Farthest-first Traversal: {accuracy_farthest}")

Epoch 2: 100%|██████████| 89/89 [00:06<00:00, 13.55it/s, loss=0.58] 


Iteration 1, Genauigkeit auf Testdaten: 0.6352


Epoch 2: 100%|██████████| 89/89 [00:06<00:00, 13.92it/s, loss=0.513]


Iteration 2, Genauigkeit auf Testdaten: 0.6944


Epoch 2: 100%|██████████| 89/89 [00:06<00:00, 13.96it/s, loss=0.643]


...

Epoch 2: 100%|██████████| 89/89 [00:06<00:00, 13.30it/s, loss=0.0508]


Iteration 9, Genauigkeit auf Testdaten: 0.6704


Epoch 2: 100%|██████████| 89/89 [00:06<00:00, 13.16it/s, loss=0.0476] 


Iteration 10, Genauigkeit auf Testdaten: 0.6976


KeyboardInterrupt: 

## Vergleiche die Genauigkeiten aller Methoden

In [ ]:
print(f"Genauigkeit ohne Active Learning: {test_accuracy}")
print(f"Genauigkeit mit Entropie-Unsicherheit: {accuracy_entropy}")
print(f"Genauigkeit mit Konfidenz-Unsicherheit: {accuracy_confidence}")
print(f"Genauigkeit mit Cluster-based Sampling: {accuracy_cluster}")
print(f"Genauigkeit mit Farthest-first Traversal: {accuracy_farthest}")

## manuelle daten labelung 
.. aus dem ersten Versuch - hab dann bestehende labels genutzt weil Zeit :)

In [19]:
# from PIL import Image, ImageOps
# import torch
# from torch.utils.data import DataLoader, Subset
# import pandas as pd
# from torchvision import transforms
# from IPython.display import display, clear_output
# 
# # Funktion zur manuellen Beschriftung der Bilder mit größerer Bildanzeige
# def manual_labeling(data_loader, indices):
#     # Leeres DataFrame für gelabelte Daten
#     labeled_data = pd.DataFrame(columns=['label'])
# 
#     for i, (images, _) in enumerate(data_loader):
#         img = transforms.ToPILImage()(images.squeeze(0))
#         img = img.resize((256, 256), Image.Resampling.LANCZOS)
#         display(img)  # Verwende display anstatt img.show()
#         label = input("Label eingeben (0 für Katze, 1 für Hund): ")
#         clear_output(wait=True)
#         labeled_data.loc[i] = [int(label)]
# 
#     return labeled_data
# 
# # Bereite den DataLoader vor, der nur die unsicheren Daten enthält
# uncertain_data_loader = Subset(ungelabeled_loader.dataset, uncertain_indices)
# uncertain_loader = DataLoader(uncertain_data_loader, batch_size=1, shuffle=False)
# 
# # Labeln der unsicheren Daten manuell
# labeled_uncertain_data = manual_labeling(uncertain_loader, uncertain_indices)
# labeled_uncertain_data.to_csv('labeled_uncertain_data.csv', index=False)